# Tie contact 3d

Author(s): Jukka Aho

## Solid block

<img src="http://results.juliafem.org/2015-12-13-box_geom.png" width=300px style="float:left">
<img src="http://results.juliafem.org/2015-12-13-box_bcs.png" width=300px style="float:left">

In [1]:
using JuliaFEM
using JuliaFEM.Preprocess: parse_aster_med_file
using JuliaFEM.Core: LinearElasticityProblem, DirichletProblem, get_connectivity, Quad4, Hex8, LinearSolver

In [2]:
mesh = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/mesh.med")
#mesh = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/twoelem_box.med")

INFO: Found 4 element sets: SYM23, SYM12, SYM13, LOAD


Dict{ASCIIString,Any} with 2 entries:
  "nodes"        => Dict(2=>[0.0,0.0,0.0],11=>[0.0,0.3333333333333333,1.0],39=>…
  "connectivity" => Dict(68=>(:QU4,:OTHER,[45,47,48,46]),2=>(:SE2,:OTHER,[9,10]…

In [3]:
# interior elements are of type HE8
field_problem = LinearElasticityProblem()
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    push!(field_problem, element)
end
# Neumann boundary condition, traction force -100 on Z direction for element set LOAD
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    (eltype == :QU4) && (elset == :LOAD) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    element["displacement traction force"] = Vector{Float64}[[0.0, 0.0, -100.0] for i=1:4]
    push!(field_problem, element)
end
info("created $(length(field_problem.elements)) elements.")

INFO: created 36 elements.


In [4]:
# boundary conditions
boundary_problem = DirichletProblem("displacement", 3)
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    (eltype == :QU4) && (elset in [:SYM23, :SYM12, :SYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    if elset == :SYM23
        element["displacement 1"] = 0.0
    elseif elset == :SYM12
        element["displacement 3"] = 0.0
    elseif elset == :SYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end
info("created $(length(boundary_problem.elements)) boundary elements.")

INFO: created 27 boundary elements.


In [5]:
ls = LinearSolver("block")
push!(ls, field_problem)
push!(ls, boundary_problem)
norm = call(ls, 0.0)

INFO: solving displacement problem, 3 dofs / nodes
INFO: solved problem in 2.11 seconds.


0.587944735792132

In [6]:
nid = 0
for (nid, coords) in mesh["nodes"]
    if isapprox(coords, [1.0, 1.0, 1.0])
        info("nid near corner = $nid")
        break
    end
end
nid

INFO: nid near corner = 7


7

In [7]:
using JuliaFEM.Test
known_value = [1/36, 1/36, -1/9]
for element in field_problem.elements
    i = indexin([nid], get_connectivity(element))[1]
    i != 0 || continue
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    info("displacement X = $(X[i]), u = $(u[i])")
    @test isapprox(-u[i], known_value)
end

INFO: displacement X = [1.0,1.0,1.0], u = [-0.02777777777777779,-0.027777777777777773,0.11111111111111113]
INFO: displacement X = [1.0,1.0,1.0], u = [-0.02777777777777779,-0.027777777777777773,0.11111111111111113]


In [8]:
xdoc, xmodel = JuliaFEM.Postprocess.xdmf_new_model()
coll = JuliaFEM.Postprocess.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.Postprocess.xdmf_new_grid(coll; time=0.0)

Xg = Dict{Int64, Vector{Float64}}()
ug = Dict{Int64, Vector{Float64}}()
for element in field_problem.elements
    conn = get_connectivity(element)
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    for (i, c) in enumerate(conn)
        Xg[c] = X[i]
        ug[c] = u[i]
    end
end
perm = sort(collect(keys(Xg)))
nodes = Vector{Float64}[Xg[i] for i in perm]
disp = Vector{Float64}[ug[i] for i in perm]
elements = []
for el in field_problem.elements
    isa(el, JuliaFEM.Core.Element{JuliaFEM.Core.Hex8}) || continue
    push!(elements, (:Hex8, get_connectivity(el)))
end
#elements
JuliaFEM.Postprocess.xdmf_new_mesh!(grid, nodes, elements)
JuliaFEM.Postprocess.xdmf_new_nodal_field!(grid, "displacement", disp)
JuliaFEM.Postprocess.xdmf_save_model(xdoc, "/tmp/foobar2.xmf");

INFO: XDFM: ndim = 192


<img src="http://results.juliafem.org/2015-12-13-box_results.png" width=300px style="float:left">

In [9]:
reshape(full(JuliaFEM.Core.assemble(field_problem, 0.0).force_vector), 3, 56)

3x56 Array{Float64,2}:
  0.0      0.0   0.0      0.0   0.0      0.0  …    0.0       0.0       0.0   
  0.0      0.0   0.0      0.0   0.0      0.0       0.0       0.0       0.0   
 -2.77778  0.0  -2.77778  0.0  -2.77778  0.0     -11.1111  -11.1111  -11.1111

## Block divived to two parts

<img src="http://results.juliafem.org/2015-12-13-divided-block-geometry.png" width=400px style="float:left">
<img src="http://results.juliafem.org/2015-12-13-divided-block-both-parts.png" width=400px style="float:left">

In [1]:
using JuliaFEM
using JuliaFEM.Preprocess: parse_aster_med_file
using JuliaFEM.Core: LinearElasticityProblem, DirichletProblem, get_connectivity, Quad4, Hex8, LinearSolver
mesh_lower = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/LOWER_BLOCK.med")
mesh_upper = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/UPPER_BLOCK.med")

INFO: Found 4 element sets: LSYM23, LSYM12, LSYM13, LOWER_BLOCK_TO_UPPER_BLOCK
INFO: Found 4 element sets: USYM23, UPPER_BLOCK_TO_LOWER_BLOCK, USYM13, LOAD


Dict{ASCIIString,Any} with 2 entries:
  "nodes"        => Dict(2=>[0.0,0.0,1.0],11=>[0.0,0.6666666666666666,1.0],39=>…
  "connectivity" => Dict(68=>(:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[42,14,4,26]),2=…

In [2]:
sort(collect(keys(mesh_lower["nodes"])))'

1x75 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  …  67  68  69  70  71  72  73  74  75

In [3]:
sort(collect(keys(mesh_upper["nodes"])))'

1x48 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  …  40  41  42  43  44  45  46  47  48

Need to renumber nodes.

In [4]:
new_node_numbering = Dict{Int64, Int64}()
upper_node_ids = sort(collect(keys(mesh_upper["nodes"])))
for (k, old_node_id) in enumerate(upper_node_ids)
    new_node_numbering[old_node_id] = 75+k
end
new_node_numbering

Dict{Int64,Int64} with 48 entries:
  2  => 77
  11 => 86
  39 => 114
  46 => 121
  25 => 100
  42 => 117
  29 => 104
  8  => 83
  20 => 95
  14 => 89
  31 => 106
  33 => 108
  18 => 93
  26 => 101
  35 => 110
  17 => 92
  44 => 119
  4  => 79
  37 => 112
  45 => 120
  13 => 88
  30 => 105
  1  => 76
  47 => 122
  32 => 107
  ⋮  => ⋮

In [5]:
function renumber!(mesh, node_numbering)
    old_nodes = mesh["nodes"]
    new_nodes = typeof(old_nodes)()
    for (node_id, node_coords) in old_nodes
        new_node_id = node_numbering[node_id]
        new_nodes[new_node_id] = node_coords
    end
    mesh["nodes"] = new_nodes
    for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
        new_elcon = [node_numbering[node_id] for node_id in elcon]
        mesh["connectivity"][elid] = (eltype, elset, new_elcon)
    end
    return mesh
end

renumber!(mesh_upper, new_node_numbering)
sort(collect(keys(mesh_upper["nodes"])))'

1x48 Array{Int64,2}:
 76  77  78  79  80  81  82  83  84  85  …  117  118  119  120  121  122  123

In [6]:
mesh_upper["connectivity"]

Dict{Int64,Tuple{Symbol,Symbol,Array{Int64,1}}} with 92 entries:
  68 => (:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[117,89,79,101])
  2  => (:SE2,:OTHER,[84,77])
  89 => (:HE8,:OTHER,[121,123,115,114,118,119,103,102])
  11 => (:SE2,:OTHER,[80,90])
  39 => (:QU4,:USYM13,[80,90,106,93])
  46 => (:QU4,:LOAD,[95,108,109,96])
  85 => (:HE8,:OTHER,[107,120,116,94,106,121,114,93])
  25 => (:SE2,:OTHER,[83,99])
  55 => (:QU4,:OTHER,[96,112,113,95])
  42 => (:QU4,:USYM13,[90,81,92,106])
  66 => (:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[94,76,88,116])
  58 => (:QU4,:OTHER,[112,100,101,113])
  29 => (:SE2,:OTHER,[101,79])
  59 => (:QU4,:OTHER,[113,101,79,87])
  8  => (:SE2,:OTHER,[76,88])
  74 => (:QU4,:OTHER,[119,103,83,99])
  90 => (:HE8,:OTHER,[105,87,79,89,122,113,101,117])
  57 => (:QU4,:OTHER,[99,83,100,112])
  20 => (:SE2,:OTHER,[95,96])
  78 => (:HE8,:OTHER,[85,86,105,104,110,108,122,120])
  14 => (:SE2,:OTHER,[91,92])
  31 => (:SE2,:OTHER,[102,103])
  70 => (:QU4,:OTHER,[97,118,119,98])
  33 => (:QU4,:

In [7]:
# interior elements are of type HE8
field_problem = LinearElasticityProblem()

for (elid, (eltype, elset, elcon)) in mesh_lower["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh_lower["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    info("lower: add element with connectivity $elcon")
    push!(field_problem, element)
end

for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    info("upper: add element with connectivity $elcon")
    push!(field_problem, element)
end

# Neumann boundary condition, traction force -100 on Z direction for element set LOAD
for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    (eltype == :QU4) && (elset == :LOAD) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    element["displacement traction force"] = Vector{Float64}[[0.0, 0.0, 100.0] for i=1:4]
    push!(field_problem, element)
end
info("created $(length(field_problem.elements)) elements.")

INFO: lower: add element with connectivity [40,69,57,23,17,64,34,5]
INFO: lower: add element with connectivity [73,54,31,61,74,53,32,62]
INFO: lower: add element with connectivity [75,52,33,63,66,30,8,36]
INFO: lower: add element with connectivity [71,74,62,59,72,75,63,60]
INFO: lower: add element with connectivity [46,47,71,68,49,50,72,69]
INFO: lower: add element with connectivity [49,50,72,69,24,25,65,64]
INFO: lower: add element with connectivity [20,43,67,42,19,46,68,41]
INFO: lower: add element with connectivity [9,37,14,1,42,67,55,21]
INFO: lower: add element with connectivity [44,45,73,70,47,48,74,71]
INFO: lower: add element with connectivity [10,11,38,37,43,44,70,67]
INFO: lower: add element with connectivity [39,13,4,16,73,54,31,61]
INFO: lower: add element with connectivity [74,53,32,62,75,52,33,63]
INFO: lower: add element with connectivity [37,38,15,14,67,70,58,55]
INFO: lower: add element with connectivity [70,73,61,58,71,74,62,59]
INFO: lower: add element with connectiv

In [8]:
# boundary conditions
boundary_problem = DirichletProblem("displacement", 3)

for (elid, (eltype, elset, elcon)) in mesh_lower["connectivity"]
    (eltype == :QU4) && (elset in [:LSYM23, :LSYM12, :LSYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_lower["nodes"][i] for i in get_connectivity(element)]
    if elset == :LSYM23
        element["displacement 1"] = 0.0
    elseif elset == :LSYM12
        element["displacement 3"] = 0.0
    elseif elset == :LSYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end

for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    (eltype == :QU4) && (elset in [:USYM23, :USYM12, :USYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    if elset == :USYM23
        element["displacement 1"] = 0.0
    elseif elset == :USYM12
        element["displacement 3"] = 0.0
    elseif elset == :USYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end

info("created $(length(boundary_problem.elements)) boundary elements.")

INFO: created 44 boundary elements.


### Creating tie contact
- define slave element surface (the one where integration happend)
- define potential master elements for slave elements

In [9]:
using JuliaFEM.Core: Element, MortarProblem, calculate_normal_tangential_coordinates!

mortar_side = mesh_upper
nonmortar_side = mesh_lower

master_elements = JuliaFEM.Core.Element[]
for (elid, (eltype, elset, elcon)) in mortar_side["connectivity"]
    (eltype == :QU4) && (elset == :UPPER_BLOCK_TO_LOWER_BLOCK) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mortar_side["nodes"][i] for i in get_connectivity(element)]
    push!(master_elements, element)
end

contact_problem = MortarProblem("displacement", 3)
for (elid, (eltype, elset, elcon)) in nonmortar_side["connectivity"]
    (eltype == :QU4) && (elset == :LOWER_BLOCK_TO_UPPER_BLOCK) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[nonmortar_side["nodes"][i] for i in get_connectivity(element)]
    element["master elements"] = master_elements
    calculate_normal_tangential_coordinates!(element, 0.0)
    push!(contact_problem, element)
end

In [10]:
using JuliaFEM.Core: DirectSolver
solver = DirectSolver()
solver.name = "tie_contact_3d"
solver.method = :UMFPACK
solver.max_iterations = 1
solver.dump_matrices = true
push!(solver, field_problem)
push!(solver, boundary_problem)
push!(solver, contact_problem)
call(solver, 0.0)

INFO: # of field problems: 1
INFO: # of boundary problems: 2
INFO: Starting iteration 1
INFO: Assembling field problems...
INFO: Assembling body 1...
INFO: dim = 369
INFO: Assembling boundary problems...
INFO: Assembling boundary 1...
INFO: Assembling boundary 2...
INFO: dumping matrices to disk, file = matrices_tie_contact_3d_host_1_iteration_1.jld
INFO: Solving system
INFO: UMFPACK: solved in 0.1845378875732422 seconds. norm = 0.37585220926688956
INFO: timing info for non-linear iteration:


(1,false)

INFO: boundary assembly       : 1.6115069389343262
INFO: field assembly          : 1.110853910446167
INFO: dump matrices to disk   : 1.0786750316619873
INFO: solve problem           : 0.3312990665435791
INFO: update element data     : 0.022845029830932617
INFO: non-linear iteration    : 4.155207872390747
INFO: Warning: did not coverge in 1 iterations!


In [11]:
using JuliaFEM.Test

nid = 0
for (nid, coords) in mesh_upper["nodes"]
    if isapprox(coords, [1.0, 1.0, 1.0])
        info("nid near corner = $nid")
        break
    end
end
nid

known_value = [1/36, 1/36, -1/9]

for element in field_problem.elements
    i = indexin([nid], get_connectivity(element))[1]
    i != 0 || continue
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    info("displacement X = $(X[i]), u = $(u[i])")
    @test isapprox(-u[i], known_value)
end

INFO: nid near corner = 82
INFO: displacement X = [1.0,1.0,1.0], u = [0.022782767229558416,0.022002026379764242,-0.08215452308106556]


LoadError: LoadError: test failed: isapprox(-(u[i]),known_value)
 in expression: isapprox(-(u[i]),known_value)
while loading In[11], in expression starting on line 14

In [12]:
xdoc, xmodel = JuliaFEM.Postprocess.xdmf_new_model()
coll = JuliaFEM.Postprocess.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.Postprocess.xdmf_new_grid(coll; time=0.0)

Xg = Dict{Int64, Vector{Float64}}()
ug = Dict{Int64, Vector{Float64}}()
for element in field_problem.elements
    conn = get_connectivity(element)
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    for (i, c) in enumerate(conn)
        Xg[c] = X[i]
        ug[c] = u[i]
    end
end
perm = sort(collect(keys(Xg)))
nodes = Vector{Float64}[Xg[i] for i in perm]
disp = Vector{Float64}[ug[i] for i in perm]
elements = []
for el in field_problem.elements
    isa(el, JuliaFEM.Core.Element{JuliaFEM.Core.Hex8}) || continue
    push!(elements, (:Hex8, get_connectivity(el)))
end
#elements
JuliaFEM.Postprocess.xdmf_new_mesh!(grid, nodes, elements)
JuliaFEM.Postprocess.xdmf_new_nodal_field!(grid, "displacement", disp)
JuliaFEM.Postprocess.xdmf_save_model(xdoc, "/tmp/foobar3.xmf");

INFO: XDFM: ndim = 369
